In [1]:
import cv2
import matplotlib.pyplot as plt
import skimage.feature
import pandas as pd
%matplotlib inline

This is the notebook for the actual extraction of the patches of the first network we trained, for each of the 2 classes we used 2 .csv files: one for the train set, that will be later split in training phase to obtain the validation set, and one for the test set.

In [2]:
# Read coordinates from files
sealions_df_train = pd.read_csv('./sealions_train.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
sealions_df_test = pd.read_csv('./sealions_test.csv', dtype={"coord_x": int, "coord_y": int, "class": str, "filename": str})
empty_df_train = pd.read_csv('./empty_train.csv', dtype={"coord_x": int, "coord_y": int, "filename": str})
empty_df_test = pd.read_csv('./empty_test.csv', dtype={"coord_x": int, "coord_y": int, "filename": str})

The function to extract the sea lions implemented an additional control to avoid patches that exceeded the borders of the image

In [3]:
# Load images and extract 96x96 patches for sea lions
def extract_sea_lions(sea_lions_df, path):
    i = 0
    file_names = sea_lions_df.filename.unique()
    for file in file_names:
        image = cv2.imread("./kaggle_sea_lions/Train/" + file)
        df = sea_lions_df[sea_lions_df['filename'] == file]
        for row in df.iterrows():
            x = row[1]['coord_x']
            y = row[1]['coord_y']
            if x < 48:
                x = 0
            elif x > len(image[0]) - 48:
                x = len(image[0]) - 96
            else:
                x = x - 48
            if y < 48:
                y = 0
            elif y > len(image) - 48:
                y = len(image) - 96
            else:
                y = y - 48
            patch = image[y:y+96, x:x+96, :]
            cv2.imwrite("./patches/" + path + "/" + str(i) + ".jpg", patch)
            i = i + 1

In [4]:
# Load images and extract 96x96 patches for background
def extract_background(empty_df, path):
    i = 0
    file_names = empty_df.filename.unique()
    for file in file_names:
        image = cv2.imread("./kaggle_sea_lions/Train/" + file)
        df = empty_df[empty_df['filename'] == file]
        for row in df.iterrows():
            x = row[1]['coord_x']
            y = row[1]['coord_y']
            patch = image[y-48:y+48, x-48:x+48, :]
            cv2.imwrite("./patches/" + path + "/" + str(i) + ".jpg", patch)
            i = i + 1

In [5]:
# Extract patches
extract_sea_lions(sealions_df_train, "sealions_train")
extract_sea_lions(sealions_df_test, "sealions_test")
extract_background(empty_df_train, "background_train")
extract_background(empty_df_test, "background_test")

N.B. since the first training used directly a balanced dataset 